In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import  pyspark.sql.functions as F
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType
from pyspark.sql import Window

In [3]:
transactions_data_path = "../data/transactions.csv"
catalogue_path = "../data/catalogue.json"
test_path = "../data/test_users.json"
ratings_path = "../data/ratings.csv"
bookmarks_path = "../data/bookmarks.csv"

In [4]:
spark = SparkSession.builder \
            .appName("geneRating") \
            .getOrCreate()

### Bookmarks exploration

In [5]:
bm_df = spark.read.csv(bookmarks_path, header=True)

In [6]:
bm_df = bm_df.withColumn("bookmark", F.lit(1))

### Transactions exploration

Загружаем данные по транзакциям

In [7]:
tr_df = spark.read.csv(transactions_data_path, header=True)

### Загружаем каталог

In [8]:
with open(catalogue_path, "r") as f:
    cat_df = json.load(f)

cat_df = pd.DataFrame.from_dict(cat_df, orient='index')

cat_df = cat_df.reset_index()

cat_df.columns = ['element_uid'] + list(cat_df.columns[1:])

cat_df.element_uid = cat_df.element_uid.astype('int64')

In [9]:
catalogue = spark.createDataFrame(cat_df)

### Data preprocessing

Добавляем к транзакциям информацию о фильме и выбрасываем транзакции по фильмам, по которым нет информации.

In [ ]:
tr_df.count()

In [ ]:
tr_df.show()

Будем использовать только информацию о типе фильма и о его длительности

In [10]:
catalogue = catalogue.select(['element_uid', 'type', 'duration'])

In [11]:
df = tr_df.join(catalogue, on='element_uid', how='inner')

In [ ]:
df.count()

Видим, что транзакций по неизвестным фильмам нет

In [ ]:
df.show(5)

In [12]:
df = df.withColumn("watched_time", F.col("watched_time")/60)

Заметим, что 1% представленного каталога имеет длительность фильма = 0

In [ ]:
catalogue.where(catalogue['duration'] == 0).count() / catalogue.count() * 100

Заменим эту длительность на 75%й квантиль времени просмотра этих фильмов

Так как неизвестно, сколько частей в многосерийном фильме и сериале, для каждого такого фильма и сериала посчитаем 75-квантиль и возьмем его за длительность фильма/сериала. Предварительно переведем длительность просмотра в минуты.

In [13]:
quantiles = df.filter((df['type'] == "multipart_movie") | (df['type'] == "series") | (df['duration'] == 0))\
    .groupBy('element_uid').agg(F.expr("percentile_approx(watched_time, 0.75)").alias('true_duration'))

In [14]:
df = df.join(quantiles, on="element_uid", how="left")

In [15]:
df = df.withColumn("true_duration", F.coalesce("true_duration", "duration"))

Добавим колонку с информацией о том, какую часть фильма посмотрел пользователь, предварительно удалив записи, по которым неизвестна длительность просмотра (около 100к)

In [16]:
df = df.where(df['watched_time'].isNotNull())\
    .withColumn("watch_part", F.col('watched_time') / F.col('true_duration'))

Посмотрим, какую часть всех фильмов составляют фильмы с watch_part > 1

In [ ]:
multiwatch_films = df.where((df['type'] == 'movie') & (df['watch_part'] > 1))
films = df.where(df['type'] == 'movie')

print(f"Multiwatch films are {multiwatch_films.count()/films.count() * 100} perc")

Считаем, что если длительность просмотра > 1,то пользователь просмотрел фильм/сериал несколько раз. Пока не будем это учитывать и заменим зачение на 1

In [17]:
df = df.withColumn("true_watch_part", F.when(df['watch_part'] > 1, 1).otherwise(F.col("watch_part")))

In [18]:
#запись в файл
default_cols = ['element_uid', 'user_uid', 'consumption_mode', 'ts', 'device_type', 'device_manufacturer', 'type',
              'true_duration', 'true_watch_part']

def write_to_prq(df, default_cols, path="../data/ratings_prq"):
    df.select(default_cols)\
        .repartition(100)\
        .write.parquet(path)

### Ratings

In [19]:
ratings_df = spark.read.csv(ratings_path, header=True)

In [ ]:
ratings_df.show(5)

In [20]:
df = df.join(ratings_df.select(['user_uid', 'element_uid', 'rating']),
             on=['user_uid', 'element_uid'], how='left')\
        .join(bm_df.select(['user_uid', 'element_uid', 'bookmark']),
             on=['user_uid', 'element_uid'], how='left')

In [21]:
df = df.fillna(0, subset=["bookmark"])

In [22]:
df = df.select([
    'user_uid',
    'element_uid',
    'consumption_mode',
    'ts',
    'type',
    'true_duration',
    'true_watch_part',
    'rating',
    'bookmark'
])

### StepDan evristic

In [23]:
df = df.withColumn('rating', F.col('rating') + 1)

In [24]:
user_rating_count = df.filter(df['rating'].isNotNull())\
                        .groupBy('user_uid').agg(F.count('rating').alias("rate_count"),
                                                 F.mean('rating').alias("rate_mean"))

In [ ]:
user_rating_count.show()

In [25]:
tmp_df = df.join(user_rating_count, on='user_uid', how='left')

In [26]:
user_implicit = df.groupBy('user_uid').agg(F.mean('true_watch_part').alias('mean_implicit'))

In [27]:
tmp_df = tmp_df.join(user_implicit, on='user_uid', how='left')

In [28]:
tmp_df = tmp_df.withColumn('rate', F.when((tmp_df['rate_count'] >= 5) & (tmp_df['rating'].isNotNull()),
                                         F.col('rating')/F.col('rate_mean') * F.col('mean_implicit'))\
                                   .otherwise(F.col('true_watch_part'))
                                   
                          )

In [ ]:
tmp_df.describe('rate').show()

In [ ]:
tmp_df.where(tmp_df['rate'].isNull()).show()

In [30]:
tmp_df = tmp_df.withColumn('rate', F.when(tmp_df['rate'] > 1, 1).otherwise(F.col('rate')))

In [31]:
#запись в файл
need_cols = ['element_uid', 'user_uid', 'ts', 'rate']

write_to_prq(tmp_df, need_cols, "../data/StepDan_ratings_prq")

In [ ]:
df.where(df['rating'] == 0).show()